<center>
<img src="Stone_Soup_Icon_Final_small.png">
<h1>Stone Soup Joint Probabiliistic Data Association (JPDA) Demo</h1>
Demonstrating the capabilities of the Stone Soup JPDA filter.
</center>

Initially, we'll set up some initial import and a plotting method which we'll use later.

In [1]:
#General imports and plotting
import datetime
import numpy as np

# Plotting
import matplotlib
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (12, 8)
plt.style.use('seaborn-colorblind')

def plot_data(groundtruth_paths, detections, tracks):
    from stonesoup.types.detection import Clutter
    data = np.array([detection.state_vector for detection in detections if not isinstance(detection, Clutter)])
    if data.any():
        plt.plot(data[:,0], data[:, 1], linestyle='', marker='o')

    data = np.array([detection.state_vector for detection in detections if isinstance(detection, Clutter)])
    if data.any():
        plt.plot(data[:,0], data[:, 1], linestyle='', marker='2')

    for path in groundtruth_paths:
        data = np.array([state.state_vector for state in path])
        plt.plot(data[:, 0], data[:, 2], linestyle=':', marker='')

    from stonesoup.types.prediction import Prediction
    for track in tracks:
        if len([state for state in track.states if not isinstance(state, Prediction)]) < 2:
            continue  # Don't plot tracks with only one detection associated; probably clutter
        data = np.array([state.state_vector for state in track.states])
        plt.plot(data[:, 0], data[:, 2], linestyle='-', marker='.')
        if hasattr(track.state, 'particles'):
            data = np.array([particle.state_vector for state in track.states for particle in state.particles])
            plt.plot(data[:,0], data[:,2], linestyle='', marker=".", markersize=1, alpha=0.5)

    plt.xlabel("$x$")
    plt.ylabel("$y$")
    custom_legend = [
        matplotlib.lines.Line2D([0], [0], color='0', linestyle='', marker='o'),
        matplotlib.lines.Line2D([0], [0], color='0', linestyle='', marker='2'),
        matplotlib.lines.Line2D([0], [0], color='0', linestyle=':', marker=''),
        matplotlib.lines.Line2D([0], [0], color='0', linestyle='-', marker='.'),
        matplotlib.lines.Line2D([0], [0], color='0', linestyle='', marker='.', markersize=1),
    ]
    plt.legend(custom_legend,
               ['Detections', 'Clutter', 'Path', 'Track', 'Particles'])

Generating Data
----------------
First we'll create some models, which will be used to generate data.

This will include a 2D-position constant velocity transition model ($x$, $\dot{x}$, $y$ and $\dot{y}$) generated by combining two 1D models (this allows multiple models to be mixed and generation of *n*-dimension models).

In [2]:
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel,\
                                               ConstantVelocity
transition_model = CombinedLinearGaussianTransitionModel(
    (ConstantVelocity(1), ConstantVelocity(1)))

And a measurement model, which will map the position based detections ($x$ and $y$) to the position in the state.

In [3]:
from stonesoup.models.measurement.linear import LinearGaussian
measurement_model = LinearGaussian(
    ndim_state=4, mapping=[0, 2], noise_covar=np.diag([10, 10]))

Next we'll create a multi-target ground truth simulation in order to generate some data for testing the tracking algorithms. This utilises the *transition model* to generate the ground truth paths, initialised at random by sampling from a *Gaussian State*. A ground truth track/path at each timestamp is created at a random *birth rate* ($\lambda$ in Poisson distribution), and randomly killed by a *death probability*.

In [4]:
from stonesoup.simulator.simple import MultiTargetGroundTruthSimulator
from stonesoup.types.state import GaussianState
from stonesoup.types.array import StateVector, CovarianceMatrix

groundtruth_sim = MultiTargetGroundTruthSimulator(
    transition_model=transition_model,
    initial_state=GaussianState(
        StateVector([[0], [0], [0], [0]]),
        CovarianceMatrix(np.diag([1000000, 10, 1000000, 10]))),
    timestep=datetime.timedelta(seconds=5),
    birth_rate=0.3,
    death_probability=0.05
)

Next we'll create a detection simulator which will generate detections based on a *detection probability* about the ground truth, utilising the *measurement model*. This model will also create clutter in our defined *measurement range*.

In [5]:
from stonesoup.simulator.simple import SimpleDetectionSimulator

detection_sim = SimpleDetectionSimulator(
    groundtruth=groundtruth_sim,
    measurement_model=measurement_model,
    meas_range=np.array([[-1, 1], [-1, 1]])*5000,  # Area to generate clutter
    detection_probability=0.9,
    clutter_rate=3,
)

detections_source = detection_sim

Building JPDA Kalman tracker components
------------------------------------

With the detection data ready, we'll now build a JPDA Kalman tracker. For this we will need a Kalman predictor, which will utilise the same *transition model* we used in the ground truth simulator.

In [6]:
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)

And a PDA Kalman updater, utilising the same *measurement model* we used in the detection simulator.

In [7]:
from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

We will also need a data associator to link detections to tracks for the update step.  For information on how the JPDA filter does this, please refer to the following references:

https://pdfs.semanticscholar.org/ecc7/0452659dfb0bc0190632f3169e53f9281395.pdf
http://www.cse.psu.edu/~rtc12/CSE598C/datassocPart2.pdf

In [8]:
from stonesoup.hypothesiser.probability import PDAHypothesiser
hypothesiser = PDAHypothesiser(predictor, updater, clutter_spatial_density=detection_sim.clutter_spatial_density, prob_detect=0.9, prob_gate=0.99)

In [9]:
from stonesoup.dataassociator.probability import JPDA
data_associator = JPDA(hypothesiser, updater)

And finally a initiator to generate tracks from unassociated detections, in this case a single point initiator generating a track for every unassociated detection.

In [10]:
from stonesoup.initiator.simple import SinglePointInitiator
initiator = SinglePointInitiator(
    GaussianState(np.array([[0], [0], [0], [0]]), np.diag([10000, 100, 10000, 1000])),
    measurement_model=measurement_model)

And a deleter to remove tracks, for this demo simply based on large covariance threshold.

In [11]:
from stonesoup.deleter.simple import CovarianceBasedDeleter
deleter = CovarianceBasedDeleter(covar_trace_thresh=1E3)

Running the JPDA Kalman tracker
---------------------------
With all the components in place, we'll now construct the tracker with a multi target tracker. Since the JPDA filter is more computationally intensive than other algorithms (due to the combinatorial explosion of permutations of track/detection associations), this notebook prints the current simulation time being processed so that you can see that the algorithm is not "hanging".

In [12]:
from stonesoup.tracker.simple import MultiTargetMixtureTracker
tracker = MultiTargetMixtureTracker(
    initiator=initiator,
    deleter=deleter,
    detector=detections_source,
    data_associator=data_associator,
    updater=updater,
)

In [14]:
tracks = set()
groundtruth_paths = set()  # Store for plotting later
detections = set()  # Store for plotting later
for time, ctracks in tracker.tracks_gen():
    tracks.update(ctracks)
    detections |= tracker.detector.detections
    #groundtruth_paths = groundtruth_paths
    print("Time: ", time)

Time:  2019-02-06 23:46:18.534571
Time:  2019-02-06 23:46:23.534571
Time:  2019-02-06 23:46:28.534571
Time:  2019-02-06 23:46:33.534571
Time:  2019-02-06 23:46:38.534571
Time:  2019-02-06 23:46:43.534571
Time:  2019-02-06 23:46:48.534571
Time:  2019-02-06 23:46:53.534571
Time:  2019-02-06 23:46:58.534571
Time:  2019-02-06 23:47:03.534571
Time:  2019-02-06 23:47:08.534571
Time:  2019-02-06 23:47:13.534571
Time:  2019-02-06 23:47:18.534571
Time:  2019-02-06 23:47:23.534571
Time:  2019-02-06 23:47:28.534571
Time:  2019-02-06 23:47:33.534571
Time:  2019-02-06 23:47:38.534571
Time:  2019-02-06 23:47:43.534571
Time:  2019-02-06 23:47:48.534571
Time:  2019-02-06 23:47:53.534571
Time:  2019-02-06 23:47:58.534571
Time:  2019-02-06 23:48:03.534571
Time:  2019-02-06 23:48:08.534571
Time:  2019-02-06 23:48:13.534571
Time:  2019-02-06 23:48:18.534571
Time:  2019-02-06 23:48:23.534571
Time:  2019-02-06 23:48:28.534571
Time:  2019-02-06 23:48:33.534571
Time:  2019-02-06 23:48:38.534571
Time:  2019-02

In [15]:
plot_data(groundtruth_paths, detections, tracks);plt.title("2D Kalman tracker");
plt.xlim(-3000, 3000);
plt.ylim(-3000, 3000);

Note: Colours are random to help differentiate overlapping data.